In [24]:
import os
import time
import json
import utilities
import traceback
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [3]:
def get_driver(proxy=None, user_agent=None):
    options = Options()
    options.headless = True

    if proxy:
        options.add_argument(f'--proxy-server={proxy}')
    if user_agent:
        options.add_argument(f'--user_agent={user_agent}')

    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference('permissions.default.image', 2)
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
    browser = webdriver.Firefox(options=options, firefox_profile=firefox_profile)

    return browser

In [40]:
def get_links():
    num_pages = 1
    user_agent = utilities.get_user_agent()
    file_name = 'builds_links.txt'
    builds_links = []
    for i in range(1, num_pages+1):
        try:
            url = utilities.parse_url(page=i)
            browser = get_driver(user_agent=user_agent)
            browser.get(url)
            time.sleep(randint(2, 5))
            soup = BeautifulSoup(browser.page_source, 'html.parser')

            # Get the link of all build cards in a single page
            builds_links_el = soup.find_all("a", {"class": "logGroup__target"}, href=True)
            builds_links = [a['href'] for a in builds_links_el]
            browser.close()
        except Exception as e:
            browser.close()
            print(traceback.format_exc())
            continue

    if not os.path.exists(f'./{file_name}'):
        # Save build href in a text file
        with open(f'{file_name}', 'w') as f:
            for build in builds_links:
                f.write(str(build) + '\n')
    else:
        with open(f'{file_name}', 'r+') as f:
            new_builds = []
            content = f.read()
            for build in builds_links:
                f.seek(0, 0)
                if build not in content:
                    new_builds.append(build)
                else:
                    break
            
            if new_builds:
                f.seek(0, 0)
                f.writelines(str(b + '\n') for b in new_builds)
                f.write(content)


In [44]:
if __name__ == '__main__':
    get_links()